## Introduction

The idea behind this notebook is to:
1. Build a Keras NLP `Backbone` model using the Hugging Face configuration (`config.json`). This creates a backbone model with randomly initialized weights.
2. Get the Keras NLP `Preprocessor` from Kaggle. We will later require this to run an end to end experiment.
3. Build a `CausalLM` using the backbone (randomly initialized weights) and the preprocessor.
4. Run generation using the `CausalLM` model.

## Setup and Imports

In [ ]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 25.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.2 which is incompatible.


In [ ]:
import json
from huggingface_hub import hf_hub_download
from keras_nlp.models import (
    PaliGemmaBackbone,
    PaliGemmaCausalLMPreprocessor,
    PaliGemmaCausalLM,
)

## Download the `config.json` from HF Hub

In [ ]:
model_id = "google/paligemma-3b-pt-224"
hf_config_file = hf_hub_download(model_id, "config.json")
with open(hf_config_file) as f:
    transformers_config = json.load(f)

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

In [ ]:
text_config = transformers_config["text_config"]
vision_config = transformers_config["vision_config"]

## Build the KerasNLP Backbone using the HF Configuration

Here one needs to go back and forth between the configurations from Kaggle and Hugging Face. For example I had to refer to [Kaggle Config](https://www.kaggle.com/models/google/paligemma?select=config.json) and [Hugging Face Config](https://huggingface.co/google/paligemma-3b-pt-224/blob/main/config.json) for the PaliGemma model.

In [ ]:
backbone = PaliGemmaBackbone(
    vocabulary_size=transformers_config["image_token_index"],
    image_size=(
        vision_config["image_size"]
        if "image_size" in vision_config.keys()
        else 224
    ),
    num_layers=text_config["num_hidden_layers"],
    num_query_heads=text_config["num_attention_heads"],
    num_key_value_heads=text_config["num_key_value_heads"],
    hidden_dim=text_config["hidden_size"],
    intermediate_dim=text_config["intermediate_size"] * 2,
    head_dim=text_config["num_image_tokens"],
    vit_patch_size=vision_config["patch_size"],
    vit_num_heads=vision_config["num_attention_heads"],
    vit_hidden_dim=vision_config["hidden_size"],
    vit_num_layers=vision_config["num_hidden_layers"],
    vit_intermediate_dim=vision_config["intermediate_size"],
)

## Get model summary

In [ ]:
backbone.summary()

Model: "pali_gemma_backbone"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ token_ids (InputLayer)    │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ images (InputLayer)       │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_embedding           │ (None, None, 2048)     │    526,647,296 │ token_ids[0][0]        │
│ (ReversibleEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pali_gemma_vit            │ (None, 224, 2048)      │    414,803,696 │ images[0][0]           │
│ (PaliGemmaVit)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply (Multiply)       │ (None, None, 2048)     │              0 │ token_embedding[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, None, 2048)     │              0 │ pali_gemma_vit[0][0],  │
│                           │                        │                │ multiply[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ padding_mask (InputLayer) │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ response_mask             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_block_0           │ (None, None, 2048)     │    110,104,576 │ concatenate[0][0],     │
│ (PaliGemmaDecoderBlock)   │                        │                │ padding_mask[0][0],    │
│                           │                        │                │ response_mask[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_block_1           │ (None, None, 2048)     │    110,104,576 │ decoder_block_0[0][0], │
│ (PaliGemmaDecoderBlock)   │                        │                │ padding_mask[0][0],    │
│                           │                        │                │ response_mask[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_block_2           │ (None, None, 2048)     │    110,104,576 │ decoder_block_1[0][0], │
│ (PaliGemmaDecoderBlock)   │                        │                │ padding_mask[0][0],    │
│                           │                        │                │ response_mask[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_block_3           │ (None, None, 2048)     │    110,104,576 │ decoder_block_2[0][0], │
│ (PaliGemmaDecoderBlock)   │                        │                │ padding_mask[0][0],    │
│                           │                        │                │ response_mask[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_block_4           │ (None, None, 2048)     │    110,104,576 │ decoder_block_3[0][0], │
│ (PaliGemmaDecoderBlock)   │                        │                │ padding_mask[0][0],    │
│                      

 Total params: 2,923,335,408 (10.89 GB)

 Trainable params: 2,923,335,408 (10.89 GB)

 Non-trainable params: 0 (0.00 B)

## Build the CausalLM

Here we first fetch the Preprocessor from Kaggle, and then use the previously built Backbone and the Preprocessor to instantiate our CausalLM.

In [ ]:
import os
from google.colab import userdata

os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")

In [ ]:
processor = PaliGemmaCausalLMPreprocessor.from_preset(
    "pali_gemma_3b_224"
)

causal_lm = PaliGemmaCausalLM(
    preprocessor=processor,
    backbone=backbone,
)

100%|██████████| 410/410 [00:00<00:00, 394kB/s]
100%|██████████| 4.07M/4.07M [00:01<00:00, 2.84MB/s]


## Run an End2End Generation

In [ ]:
import numpy as np

image = np.random.rand(224, 224, 3)
prompt = ["where is the cow standing?"]

In [ ]:
outputs = causal_lm.generate({
    "images": image,
    "prompts": prompt,
})

You will notice that the outputs are gibberish, this is because the backbone has randomly initialized weights!

In [ ]:
outputs

['where is the cow standing? forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested forested